In [190]:
import requests
import pandas

Actions = ["http://127.0.0.1:8000/actions/actions", "http://127.0.0.1:8000/actions/queue"]
payload = ""
headers = {}

In [193]:

import requests
import pandas
import json
import ast

class readURL:
    def __init__(self, url: str):
        self.url = url
        
    def read(self) -> requests:
        payload = ""
        headers = {}
        response = requests.request("GET", self.url, headers=headers, data=payload)
        return response
    
    def json(self) -> json:
        return self.read().text

    def only_dict(d):
        return ast.literal_eval(d)

    def list_of_dicts(ld):
        return dict([(list(d.values())[1], list(d.values())[0]) for d in ast.literal_eval(ld)])

    def expand(self, data:pandas.DataFrame, withId:bool = True) -> pandas.DataFrame:
        for x in data.columns:
            if data[x].dtypes == object and data[x].name != '_id' :
                if withId:
                    newColumn = pandas.json_normalize(data[x].apply(ast.literal_eval).tolist()).add_prefix(str(data[x].name) + '.')
                else:
                    newColumn = pandas.json_normalize(data[x].apply(ast.literal_eval).tolist()).drop(columns=['_id']).add_prefix(str(data[x].name) + '.')
                data = data.drop(columns=data[x].name)
                data = data.join([newColumn])
        return data

    def dataFrame(self, withId:bool = True, columnExpand:bool = True) -> pandas.DataFrame:
        data = pandas.read_json(self.json())
        if not withId:
            data = data.drop(columns=['_id'])
        if columnExpand:
            data = self.expand(data=data, withId=withId)
            return data
        else:
            return data
    

class readJSON:
    def __init__(self, json: json):
        self.json = json

    def only_dict(d):
        return ast.literal_eval(d)

    def list_of_dicts(ld):
        return dict([(list(d.values())[1], list(d.values())[0]) for d in ast.literal_eval(ld)])

    def expand(self, data:pandas.DataFrame, withId:bool = True) -> pandas.DataFrame:
        for x in data.columns:
            if data[x].dtypes == object and data[x].name != '_id' :
                if withId:
                    newColumn = pandas.json_normalize(data[x].apply(ast.literal_eval).tolist()).add_prefix(str(data[x].name) + '.')
                else:
                    newColumn = pandas.json_normalize(data[x].apply(ast.literal_eval).tolist()).drop(columns=['_id']).add_prefix(str(data[x].name) + '.')
                data = data.drop(columns=data[x].name)
                data = data.join([newColumn])
        return data

    def dataFrame(self, withId:bool = True, columnExpand:bool = True) -> pandas.DataFrame:
        data = pandas.read_json(self.json)
        if not withId:
            data = data.drop(columns=['_id'])
        if columnExpand:
            data = self.expand(data=data, withId=withId)
            return data
        else:
            return data
    


In [191]:
queue = readURL("http://127.0.0.1:8000/battery/battery")
df = queue.dataFrame(withId=False)
df

,dateTime,Sensor.Voltage,Sensor.Current,Sensor.Temperature,Calculate.Status,Calculate.Percent
0,2022-12-06 09:07:46.676000+00:00,1.0,None,None,None,0.4
1,2022-12-06 09:08:30.996000+00:00,1.0,None,None,None,0.4
2,2022-12-06 09:38:17.860000+00:00,1.0,None,None,None,0.4


In [195]:
json_ = readJSON(queue.json())
json_.dataFrame()

TypeError: 'str' object is not callable